In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
img_id = "CA184632V0343"
overwrite = False

output_fld = "/data_1/ATM/data_1/georef"


<h4>Check if the image is already geo-referenced</h4>


In [3]:
import pandas as pd

# define the path to processed images
csv_path = output_fld + "/img_processed_images.csv"

# load processed images as dict
processed_images = pd.read_csv(csv_path, delimiter=";")
processed_images.set_index('id', inplace=True)
processed_images = processed_images.to_dict(orient='index')

# check if image is already geo-referenced
if processed_images.get(img_id, {}).get('status') == "georeferenced" and not overwrite:
    print(f"{img_id} already processed")
    exit()

<h4>Find overlapping images</h4>

In [4]:
import os
import glob
import src.base.find_overlapping_images as foi

# define path to the already geo-referenced images
path_satellite_fld = "/data_1/ATM/data_1/georef/sat"

# get the filenames of all tiffs in this folder
# List all .tif files in the folder
tif_files = glob.glob(os.path.join(path_satellite_fld, '*.tif'))
tif_files = [os.path.splitext(os.path.basename(file))[0] for file in tif_files]

# add the image_id to the list of tif_files
tif_files.append(img_id)

# find overlapping images for the image to geo-reference just based on the image id
overlapping_images = foi.find_overlapping_images(tif_files, working_modes=["ids"], important_id=img_id)

print(overlapping_images)

1846 2142
1846 1833
1846 1817
1846 1833
1846 1821
1846 1816
1846 1816
1846 1833
1846 1833
1846 1821
1846 1817
1846 1833
1846 1833
1846 1745
1846 1833
1846 1846
1846 2142
1846 1846
1846 2141
1846 2124
1846 1833
1846 1816
1846 1846
1846 2142
1846 2124
1846 1822
1846 1843
1846 1817
1846 1833
1846 1833
1846 1818
1846 1818
1846 1818
1846 1833
1846 1818
1846 1812
1846 1821
1846 1833
1846 2142
1846 1821
1846 1818
1846 1816
1846 1821
1846 1821
1846 1821
1846 1833
1846 1833
1846 1817
1846 2073
1846 1812
1846 1833
1846 2142
1846 1825
1846 1812
1846 1843
1846 1816
1846 1821
1846 2142
1846 1846
1846 1846
1846 2142
1846 2142
1846 1821
1846 1644
1846 2142
1846 2142
1846 1821
1846 1848
1846 2141
1846 1846
1846 1833
1846 1813
1846 1846
1846 1833
1846 1833
1846 2142
1846 1833
1846 2167
1846 1812
1846 1816
1846 2142
1846 1821
1846 1812
1846 1833
1846 1816
1846 1833
1846 1833
1846 1821
1846 1821
1846 1966
1846 2147
1846 1821
1846 1813
1846 2141
1846 1813
1846 1821
1846 1833
1846 1833
1846 1644
1846 2141


<h4>Load the images</h4>

In [6]:
import src.load.load_image as ld

# load image to geo-reference
image = ld.load_image(img_id)

# list of already geo-referenced images
georeferenced_images = []

# iterate through overlapping images
for g_img_id in overlapping_images[img_id]:

    # load the geo-referenced image
    georef_img = ld.load_image(g_img_id)
    georeferenced_images.append(georef_img)

<h4>Load the transforms</h4>

In [ ]:
import src.load.load_transform as lt

<h4>Create masks</h4>

In [10]:
import src.base.connect_to_database as ctd
import src.base.create_mask as cm

conn = ctd.establish_connection()

sql_string_fid_marks = f"SELECT * FROM images_fid_points WHERE image_id='{img_id}'"
data_fid_marks = ctd.execute_sql(sql_string_fid_marks, conn)

ids = [img_id] + overlapping_images[img_id]

georeferenced_masks = []

for i, _id in reversed(list(enumerate(ids))):

    sql_string_fid_marks = f"SELECT * FROM images_fid_points WHERE image_id='{_id}'"
    data_fid_marks = ctd.execute_sql(sql_string_fid_marks, conn)

    sql_string_extracted = f"SELECT * FROM images_extracted WHERE image_id='{_id}'"
    data_extracted = ctd.execute_sql(sql_string_extracted, conn)

    # Get the fid marks for the specific image_id
    fid_marks_row = data_fid_marks.loc[data_fid_marks['image_id'] == _id].squeeze()

    # Create fid mark dict using dictionary comprehension
    fid_dict = {i: (fid_marks_row[f'fid_mark_{i}_x'], fid_marks_row[f'fid_mark_{i}_y']) for i in range(1, 5)}

    # get the text boxes of the image
    text_string = data_extracted.loc[data_extracted['image_id'] == _id]['text_bbox'].iloc[0]

    if len(text_string) > 0 and "[" not in text_string:
        text_string = "[" + text_string + "]"

    # create text-boxes list
    text_boxes = [list(group) for group in eval(text_string.replace(";", ","))]

    # load the mask
    mask = cm.create_mask(image, fid_dict, text_boxes)

    if i > 0:
        georeferenced_masks.append(mask)

In [ ]:
import src.georef.georef_img as gi

georefImg = gi.GeorefImage(enhance_image=True, enhance_georef_images=True)

georefImg.georeference(image, georeferenced_images, georeferenced_transforms,
                       mask=mask, georeferenced_masks=georeferenced_masks)
